In [ ]:

 
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import hog  # Import HOG descriptor from skimage

def compute_hog_features(window):
    # Convert to grayscale if the window is not already
    if len(window.shape) == 3:
        window = cv2.cvtColor(window, cv2.COLOR_BGR2GRAY)
    
    # Compute HOG features for the window
    # Use the 'orientations', 'pixels_per_cell', and 'cells_per_block' parameters as needed
    hog_features, _ = hog(window, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')
    
    return hog_features

# Adjust these paths according to your dataset structure
image_folder = r'/home/m_roshni/SFR/SFR/color_equlsize_jpg/color_equlsize_jpg'
labels_df = pd.read_excel('/home/m_roshni/SFR/SFR/line_gt.xlsx')

# Initialize a dictionary to store HOG feature sequences for each image
word_sequences = {}

for index, row in labels_df.iterrows():
    image_name = row['image name']
    character_sequence = row['gt']
    print(image_folder, image_name)
    
    # Load the corresponding image
    image_path = os.path.join(image_folder, image_name)
    image = cv2.imread(image_path)
    
    if image is None:
        print(f"Image {image_name} could not be loaded.")
        continue

    image_width = image.shape[1]  # Get image width
    
    # Define the sliding window parameters
    window_width = 150  # Width of the sliding window in pixels
    step_size = 20     # Step size of the sliding window in pixels

    # Calculate the number of windows
    num_windows = (image_width - window_width) // step_size + 1

    # Create an empty list to hold the sequence of HOG features for this word
    word_hog_sequence = []

    # Create sliding windows and compute HOG features for each window
    for i in range(num_windows):
        # Calculate the pixel range covered by the current window
        window_start = i * step_size
        window_end = window_start + window_width
        
        # Extract the window from the image
        window = image[:, window_start:window_end]  # Assume height is all rows, width is the window
        
        # Compute HOG features for the window
        hog_features = compute_hog_features(window)
        word_hog_sequence.append(hog_features)
        
    # Store the HOG feature sequence for the current word/image
    word_sequences[image_name] = np.vstack(word_hog_sequence)  # Stack the list of feature arrays to form a matrix


/home/m_roshni/SFR/SFR/color_equlsize_jpg/color_equlsize_jpg MaI849_039_line_1.jpg
/home/m_roshni/SFR/SFR/color_equlsize_jpg/color_equlsize_jpg MaI849_039_line_2.jpg
/home/m_roshni/SFR/SFR/color_equlsize_jpg/color_equlsize_jpg MaI849_039_line_3.jpg
/home/m_roshni/SFR/SFR/color_equlsize_jpg/color_equlsize_jpg MaI849_039_line_4.jpg
/home/m_roshni/SFR/SFR/color_equlsize_jpg/color_equlsize_jpg MaI849_039_line_5.jpg
/home/m_roshni/SFR/SFR/color_equlsize_jpg/color_equlsize_jpg MaI849_039_line_6.jpg
/home/m_roshni/SFR/SFR/color_equlsize_jpg/color_equlsize_jpg MaI849_039_line_7.jpg
/home/m_roshni/SFR/SFR/color_equlsize_jpg/color_equlsize_jpg MaI849_039_line_8.jpg
/home/m_roshni/SFR/SFR/color_equlsize_jpg/color_equlsize_jpg MaI849_039_line_9.jpg
/home/m_roshni/SFR/SFR/color_equlsize_jpg/color_equlsize_jpg MaI849_038_line_1.jpg
/home/m_roshni/SFR/SFR/color_equlsize_jpg/color_equlsize_jpg MaI849_038_line_2.jpg
/home/m_roshni/SFR/SFR/color_equlsize_jpg/color_equlsize_jpg MaI849_038_line_3.jpg
/hom

In [ ]:
# Initialize lists for sequences and their corresponding lengths
sequences = []
lengths = []
labels = []

# For each image, add its DCT feature sequence and the corresponding label (word)
for image_name, dct_sequence in word_sequences.items():
    # Get the corresponding label (word) from the DataFrame
    label = labels_df.loc[labels_df['image name'] == image_name, 'gt'].values[0]
    
    # Append the DCT sequence and its length
    sequences.append(dct_sequence)
    lengths.append(len(dct_sequence))
    labels.append(label)  # Store the label for each image

# Convert sequences to a single numpy array
X = np.concatenate(sequences)  # Flatten the list of sequences

# Convert lengths to numpy array (required by hmmlearn)
model_lengths = np.array(lengths)

# Print the prepared data
print("Sequences shape:", X.shape)
print("Lengths:", model_lengths)
print("Labels:", labels[:5])  # Display the first 5 labels

In [ ]:
from hmmlearn import hmm

# Initialize the HMM model
n_states = 71  # Number of hidden states in the HMM
model = hmm.GaussianHMM(n_components=n_states, covariance_type="diag", n_iter=1000)

# Train the model
model.fit(X, model_lengths)

print("HMM training complete.")

In [ ]:
from joblib import dump

# Save the model to a file
dump(model, 'line_sequences_71_hmm_mode_11-11-2024_1000.joblib')

In [ ]:
def predict_sequence(model, image_path, window_width=150, step_size=20):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image at {image_path} could not be loaded.")

    image_width = image.shape[1]
    num_windows = (image_width - window_width) // step_size + 1
    dct_sequence = []

    # Generate DCT features for each window
    for i in range(num_windows):
        window_start = i * step_size
        window_end = window_start + window_width
        window = image[:, window_start:window_end]
        dct_features = compute_dct_features(window)
        dct_sequence.append(dct_features)

    # Predict the sequence of states
    dct_sequence = np.array(dct_sequence)
    predicted_states = model.predict(dct_sequence)
    
    return predicted_states

In [ ]:
import os
import numpy as np
import pandas as pd
from hmmlearn import hmm

# Load the HMM model (replace with your trained model)
# model = ...


# Read the Excel file containing character sequences
import pandas as pd
df = pd.read_excel('/home/m_roshni/SFR/SFR/line_gt.xlsx') 

# Extract the 'Label' column containing character sequences
character_sequences = df['gt'].tolist()

# Set to hold unique characters
unique_characters = set()

# Loop through each sequence and add characters to the set
for sequence in character_sequences:
    unique_characters.update(sequence)

# Now, 'unique_characters' contains all unique characters from the sequences
sorted_characters = sorted(unique_characters)

# Create the mapping of state index to character
state_to_char = {i: char for i, char in enumerate(sorted_characters)}

# Directory containing the images
image_folder = '/home/m_roshni/SFR/SFR/color_equlsize_jpg/color_equlsize_jpg'

# Get all image files in the folder (adjust pattern as needed)
image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]
states = []
# Process each image in the folder
for image_file in image_files:
    # Get the full path of the image
    image_path = os.path.join(image_folder, image_file)
    
    # Predict the sequence of states for the image
    predicted_states = predict_sequence(model, image_path)
    predicted_states = np.array(predicted_states) 
    states.append([image_file, predicted_states])
    print(states)
# Convert data to a DataFrame
output_df = pd.DataFrame(states, columns=['Image Name', 'Predicted States'])

# Save the DataFrame to an Excel file
output_df.to_excel('/home/m_roshni/predicted_states_line_hog.xlsx', index=False)

In [2]:
pip install --upgrade scikit-image


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install scikit-image==0.19.3


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 5.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 6.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.24.0
    Uninstalling scikit-image-0.24.0:
      Successfully uninstalled scikit-image-0.24.0
Note: you may need to restart the kernel to use updated packages.
